In [1]:
import numpy as np
import os
import sys
from fractions import gcd
from numbers import Number

import torch
from torch import Tensor, nn
from torch.nn import functional as F

from data import ArgoDataset, collate_fn
from utils import gpu, to_long,  Optimizer, StepLR

from layers import Conv1d, Res1d, Linear, LinearRes, Null
from numpy import float64, ndarray
from typing import Any, Callable, Dict, List, Optional, Tuple, Type, Union

from lanegcn import PredNet, get_model
import torch
from torch.utils.data import Sampler, DataLoader

import matplotlib.pyplot as plt

config, Dataset, collate_fn, net, loss, post_process, opt = get_model()
import os

import argparse
import numpy as np
import random
import sys
import time
import shutil
from importlib import import_module
from numbers import Number

import torch
from torch.utils.data import Sampler, DataLoader


from utils import Logger, load_pretrain
def worker_init_fn(pid):
    np_seed = int(pid)
    np.random.seed(np_seed)
    random_seed = np.random.randint(2 ** 32 - 1)
    random.seed(random_seed)


dataset = Dataset(config["train_split"], config, train=True)
train_loader = DataLoader(
    dataset,
    batch_size=config["batch_size"],
    num_workers=config["workers"],
    shuffle=False,
    collate_fn=collate_fn,
    pin_memory=True,
    worker_init_fn=worker_init_fn,
    drop_last=True,
)


for i, data in enumerate(train_loader):
    break


NameError: name 'config' is not defined

In [57]:
from lanegcn import ActorNet, MapNet, actor_gather, graph_gather
actor_net = ActorNet(config)
map_net = MapNet(config)

# construct actor feature
actors, actor_idcs = actor_gather(gpu(data["feats"]))
actor_ctrs = gpu(data["ctrs"])
actors = actor_net(actors)

# construct map features
graph = graph_gather(to_long(gpu(data["graph"])))
nodes, node_idcs, node_ctrs = map_net(graph)

In [58]:
from TNT_fun import Classifier

goda_classifier = Classifier(config)

In [59]:
goda_cls = goda_classifier(nodes)
print(nodes.size(), goda_cls.size())

torch.Size([33084, 128]) torch.Size([33084])


In [60]:
batch_size = len(graph['ctrs'])
print(len(graph['ctrs']))

32


In [61]:
DEFAULT_GODA_NUM = 200
goda_score = []
goda_coord = []
goda_feat = []
for i in range(batch_size):
    scores = goda_cls[graph['idcs'][i]]
    _, idcs = torch.sort(scores, descending=True)
    assert len(idcs) > DEFAULT_GODA_NUM, 'Invalid goda number'

    idcs = idcs[:DEFAULT_GODA_NUM]
    goda_score.append(scores[idcs])
    goda_coord.append(graph['ctrs'][i][idcs])
    goda_feat.append(nodes[graph['idcs'][i]][idcs])

In [62]:
print(len(goda_score), goda_score[0].size(), goda_coord[0].size(), goda_feat[0].size())

32 torch.Size([200]) torch.Size([200, 2]) torch.Size([200, 128])


In [63]:
goda_score = torch.stack(goda_score, dim=0)
goda_coord = torch.stack(goda_coord, dim=0)
goda_feat = torch.stack(goda_feat, dim=0)

In [64]:
print(goda_score.size())

torch.Size([32, 200])


In [65]:
from TNT_fun import GoalGenerator

goal_generator = GoalGenerator(config, n_blk=2)


goal_pred = goal_generator(goda_score, goda_coord, goda_feat)

In [66]:
goal_pred.size()

torch.Size([32, 6, 2])

if training:

In [67]:
print(len(data['ctrs']), data['ctrs'][0].size())

32 torch.Size([6, 2])


In [68]:
GT = []
for i in range(len(data['ctrs'])):
    GT.append(data['ctrs'][0])

# goal_gt = torch.cat(GT, dim=1).permute(1,0,2)
# print(goal_gt.size())

In [69]:
from TNT_fun import TrajCompletor

traj_completor = TrajCompletor(config)
goal_gt = data['ctrs']
device = torch.device('cpu')

In [70]:
goal_mock = torch.stack(GT)
print(goal_mock.size())

torch.Size([32, 6, 2])


In [71]:
agent_idcs = torch.LongTensor([idcs[0] for idcs in actor_idcs])
agent_feat = actors[agent_idcs]

In [72]:
traj_pred = traj_completor(agent_feat, goal_mock)

In [73]:
score_pred = torch.ones(traj_pred.shape[0], traj_pred.shape[1])


In [74]:
traj_pred.size()

torch.Size([32, 6, 30, 5])

In [75]:
score_pred.size()

torch.Size([32, 6])

In [76]:
score_pred = torch.ones(traj_pred.shape[0], traj_pred.shape[1])

In [77]:
score_pred.size()

torch.Size([32, 6])

IF Train = False

In [78]:
from torch_scatter import scatter_add
edges = graph_da['ms_edges'][0]
goda_cls_tmp = goda_cls.clone()
goda_cls_tmp = scatter_add(goda_cls_tmp.index_select(0, edges['u']), edges['v'], out=goda_cls_tmp, dim=0)

traj_pred = traj_completor(agent_feat, goal_pred)
traj_pred[:, :, -1, 0:2] = goal_pred

# score_pred = torch.ones(traj_pred.shape[0], traj_pred.shape[1])  # uniform distribution
# ~ score from heatmap
score_pred = []
for b in range(batch_size):
    goals = graph['ctrs'][b]
    goal_scores = goda_cls_tmp[graph['idcs'][b]]
    score_tmp = []
    for g in goal_pred[b]:
        dist = torch.sqrt(torch.sum((goals - g.unsqueeze(0))**2, dim=1))
        min_idx = torch.argmin(dist)
        score_tmp.append(goal_scores[min_idx])
    score_pred.append(score_tmp)
score_pred = torch.Tensor(score_pred).to(device)

NameError: name 'graph_da' is not defined

In [ ]:
factors = np.array([2**s for s in range(4)], dtype=np.int)
factors

array([1, 2, 4, 8])

In [79]:
from matplotlib.path import Path
from shapely.geometry import LineString, Point, Polygon, MultiPolygon
from shapely.ops import cascaded_union, nearest_points, unary_union
from argoverse.map_representation.map_api import ArgoverseMap

MAP_RADIUS = 100.0
DA_RESOLUTION = 1.0
DA_NUM_SCALES = 4
am = ArgoverseMap()




def construct_freespace_layer(city_name, orig, rot, lane_ids):
        # ~ Sample according to freespace
        x_min = np.floor(orig[0] - MAP_RADIUS)
        x_max = np.floor(orig[0] + MAP_RADIUS)
        y_min = np.floor(orig[1] - MAP_RADIUS)
        y_max = np.floor(orig[1] + MAP_RADIUS)

        x, y = np.meshgrid(np.arange(x_min, x_max, DA_RESOLUTION), np.arange(y_min, y_max, DA_RESOLUTION))
        pts = np.stack((x.flatten(), y.flatten()), axis=1)

        # Driveable area
        idcs_da = am.get_raster_layer_points_boolean(pts, city_name, 'driveable_area')  # np.ndarray, Boolean

        # lane ROI
        lane_polygon = unary_union([
            Polygon(am.get_lane_segment_polygon(lane, city_name)).buffer(0.1)
            for lane in lane_ids])
        if isinstance(lane_polygon, MultiPolygon):
            polygons = list(lane_polygon)
            for polygon in polygons:
                # select the polygon contains the target
                if polygon.contains(Point(orig)):
                    lane_polygon = polygon
        lane_polygon = lane_polygon.buffer(15.0)

        poly = Path(np.array(lane_polygon.exterior.coords.xy).T)
        idcs_roi = poly.contains_points(pts)

        idcs_da = np.logical_and(idcs_da, idcs_roi)
        pts_da = pts[idcs_da]

        # ~ get graph topology
        ids = np.full(len(pts), -1)
        npts_da = np.count_nonzero(idcs_da)
        ids[idcs_da] = np.arange(npts_da)
        ids = ids.reshape(x.shape)

        ms_edges = dict()
        factors = np.array([2**s for s in range(DA_NUM_SCALES)], dtype=np.int)
        for F in factors:
            ids_us = []  # up
            ids_ls = []  # left
            ids_uls = []  # upper-left
            ids_urs = []  # upper-right
            for K in range(1, F + 1):
                pad_n = np.ones((2, 2), dtype=np.int) * K
                ids_padded = np.pad(ids, pad_n, constant_values=-1, mode='constant')

                # ! Only get a half of the edges due to memory issue
                # ! The other part can be retrieved by the symmetry
                ids_u = ids_padded[:-K*2, K:-K].flatten()
                ids_us.append(ids_u)

                ids_l = ids_padded[K:-K, :-K*2].flatten()
                ids_ls.append(ids_l)

                ids_ul = ids_padded[:-K*2, :-K*2].flatten()
                ids_uls.append(ids_ul)

                ids_ur = ids_padded[:-K*2, K*2:].flatten()
                ids_urs.append(ids_ur)

            ids_us = np.array(ids_us)
            ids_ls = np.array(ids_ls)
            ids_uls = np.array(ids_uls)
            ids_urs = np.array(ids_urs)

            d = F - 1
            if d == 0:
                ids_tgts = [ids_us[-1], ids_ls[-1], ids_uls[-1], ids_urs[-1]]
            else:
                ids_tgts = []
                for ids_tar in [ids_us, ids_ls, ids_uls, ids_urs]:
                    ids_d = ids_tar[:d]
                    ids_fin = ids_tar[d]
                    ids_fin[np.any(ids_d == -1, axis=0)] = -1
                    ids_tgts.append(ids_fin)

            edges_dirs = []
            # ! Self-loop is not saved due to memory issue
            # edges_dirs += [np.tile(np.arange(npts_da), (2, 1)).transpose()]  # self loop
            for ids_tgt in ids_tgts:
                edges = np.stack((ids_tgt[idcs_da], np.arange(npts_da)), axis=1)
                edges = edges[edges[:, 0] >= 0]
                edges_dirs.append(edges)
            edges_dirs = np.concatenate(edges_dirs).astype(np.int16)

            edges = dict()
            edges['u'] = edges_dirs[:, 0]
            edges['v'] = edges_dirs[:, 1]
            ms_edges[F] = edges
            # print('new - level: {}, num: {}'.format(F, ms_edges[F]['u'].shape))

        pts_da[:, 0:2] = (pts_da[:, 0:2] - orig).dot(rot)

        graph = dict()
        graph['num_nodes'] = len(pts_da)
        graph['ctrs'] = pts_da.astype(np.float32)
        graph['ms_edges'] = ms_edges
        return graph

In [97]:
x_min, x_max, y_min, y_max = config['pred_range']
radius = max(abs(x_min), abs(x_max)) + max(abs(y_min), abs(y_max))
lane_ids = am.get_lane_ids_in_xy_bbox(data['orig'][0][0], data['orig'][0][1], data['city'][0], radius)

In [98]:
a =  construct_freespace_layer(data['city'][0], data['orig'][0], data['rot'][0], lane_ids)

TypeError: rsub() received an invalid combination of arguments - got (Tensor, numpy.ndarray), but expected one of:
 * (Tensor input, Tensor other, *, Number alpha)
 * (Tensor input, Number other, Number alpha)
